In [1]:
import wandb

# fenics Finite Element
from fenics import *
from mshr import *
from dolfin import *

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# typical libaries
import numpy as np
import matplotlib.pyplot as plt

#% matplotlib notebook
# from IPython.display import Image
# from IPython.display import set_matplotlib_formats
# from IPython.display import clear_output
# set_matplotlib_formats('png', 'pdf')
# get_ipython().run_line_magic('matplotlib', 'inline')

import gym
from gym import spaces

from wandb.integration.sb3 import WandbCallback

from tqdm import tqdm
from PINN_3D import PINN
from pyDOE import lhs         #Hypercube Sampling
import scipy.io


cuda


In [2]:
from wrapper import PINN_env

from environment import Convection
r = str(np.random.random())[2:8]

print(r)
sensor_coords = np.array([[0.1, -0.9],[0.9,0.9],[0.5,0.45]])
env1 = Convection(dt = 1e-2, sensor_coords = sensor_coords,
                     continuous = True,
                     noisy_IC = True,
                     norm_reward = False,
                     scale_reward = True,
                     verbose = True,
                     noisy_source = False,
                     wandb_name = f'test_standard_a2c_{r}_yesNoise_',
                     project_name = 'Convection_Cont_final',
                 Cp = 5, #10
                 k = 5,
                theta = 5)

env_pinn = PINN_env(env1, norm_obs = True,
                    standardize_obs = False,
                    discrete_bc = False, #False
                    mean_obs = False,
                   use_wandb=True,
                   max_loss = 10,
                   min_delta = 1.0,
                   max_iter = 10,
                   flow_vel = None)

553148


wandb: Currently logged in as: jacob-turner (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/jacob/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
0it [00:00, ?it/s]
0it [00:00, ?it/s]

In [3]:
from tensorforce.environments import Environment
import tensorforce

env = Environment.create(
    environment=env1)#, max_epsisode_timesteps = 200)



import tensorforce
from tensorforce.agents import Agent



a2c = Agent.create(
    agent='a2c', environment=env, 
    batch_size=100, 
    update_frequency=1.0,
    #memory = 10000,
    learning_rate=1e-4,
    max_episode_timesteps = 200,
    horizon=1, 
    discount=0.99,
    exploration=0.1
)



In [ ]:
from tensorforce.execution import Runner

runner = Runner(
    agent=a2c,
    environment=env
)

In [ ]:
runner.run(num_episodes=1000) #memory issues

In [ ]:
plt.plot(np.arange(len(env1.info['energy'])), env1.info['energy'])
plt.axhline(y = 0, color ="green", linestyle ="--")
plt.ylim(-20,20)

In [ ]:
#env_pinn.save_PINN(episode = 150, id = r)

a2c.save(directory = 'models/', 
         filename = f'standard_a2c_{r}_', 
         append="timesteps")

In [ ]:
y = torch.stack(env_pinn.info['PINN_loss']).cpu().detach().numpy()

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

y_smooth = smooth(y, 400)

plt.plot(np.arange(len(y)),y_smooth)

In [ ]:
sum_rewards = 0.0
for _ in range(1):
    states = env.reset()
    internals = a2c.initial_internals()
    terminal = False
    while not terminal:
        actions, internals = a2c.act(
            states=states, internals=internals,
            independent=True, deterministic=True
        )
        states, terminal, reward = env.execute(actions=actions)
        sum_rewards += reward
        env1.render()
        clear_output(wait=True)
print('Mean episode reward:', sum_rewards / 100)

# Close agent and environment
agent.close()
environment.close()

In [ ]:

env_pinn.save_PINN(episode = 150, id = r)

fname = f'models/diffusion_PINN_39000_SAC_fail_{r}.pth'
if True: 
    model.save(fname)
    print('saved.')

In [ ]:
obs = env_pinn.reset(ic_temp = -20)
env_pinn.test_epsisode(env = env_pinn, 
                      render = True, 
                      load_PINN = env_pinn.pinn_model, 
                      model_name = fname, 
                      model = SAC)

In [ ]:
og_reward = np.array(env1.info['reward']).copy()
plt.plot(np.arange(len(og_reward[-200:])), og_reward[-200:])

In [ ]:
#SAC 001358
            gamma = 0.99,
            tau = 0.005,
            learning_starts = 5000,
            buffer_size = 10000,
            batch_size = 1000,
            optimize_memory_usage = True,
            train_freq = 5,
            gradient_steps = 1,
            target_update_interval = 500